# Azure OpenAI

Los recursos de Azure OpenAI difieren de los recursos de OpenAI estándar, ya que no se pueden generar incrustaciones a menos que se use un modelo de incrustación. Las regiones donde están disponibles estos modelos se pueden encontrar aquí: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models#embeddings-models

Además, las regiones que admiten modelos de incrustación lamentablemente no admiten las últimas versiones (<*>-003) de los modelos de OpenAI, por lo que nos vemos obligados a usar una región para incrustaciones y otra para la generación de texto.

Necesitarás instalar LlamaIndex 🦙.

In [1]:
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [2]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.3 MB/s eta 0:00:00


In [3]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Configuramos el modelo de incrustación (para la recuperación) y llm (para la generación de texto). Tenga en cuenta que no solo necesita los nombres de los modelos (por ejemplo, "text-embedding-ada-002"), sino también los nombres de implementación del modelo (el que eligió al implementar el modelo en Azure). Debe pasar el nombre de la implementación como parámetro cuando inicialice `AzureOpenAI` y `OpenAIEmbedding`.

In [4]:
api_key = "XXXX"
azure_endpoint = "https://<--->.openai.azure.com/"
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-4o", #usar nombre del modelo
    deployment_name="gpt-4o", #usar nombre del deployment
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [5]:
# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002", #usar nombre del modelo
    deployment_name="text-embedding-ada-002", #usar nombre del deployment
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
documents = SimpleDirectoryReader(
    input_files=["prueba_txt_file.txt"]
).load_data()

ValueError: File prueba_txt_file.txt does not exist.

In [ ]:
documents

[Document(id_='bee77497-d4ee-4ea4-a18d-51731f53cc43', embedding=None, metadata={'file_path': 'prueba_txt_file.txt', 'file_name': 'prueba_txt_file.txt', 'file_type': 'text/plain', 'file_size': 910, 'creation_date': '2024-10-18', 'last_modified_date': '2024-10-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='La ciudad de Chernóbil fue evacuada el 27 de abril de 1986, un día después del accidente nuclear sucedido en la central nuclear Vladímir Ilich Lenin (a 3 kilómetros de la ciudad de Prípiat, actual Ucrania), considerado como el más grave en la Escala Internacional de Accidentes Nucleares (el accidente mayor, nivel 7). Como consecuencia , las personas tuvieron que abandonar sus hogares, personas muertas y heridas, radioactividad liberada, áreas cont

In [ ]:
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [ ]:
query = "En español, qué ocurrió en Chernóbil?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print(f"La consulta fue: {query} \n")
print(f"La respeusta fue: {answer} \n")

INFO:httpx:HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://data-tribe-openai.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
> Source (Doc id: 62ad6999-1669-485d-9341-2750f37898e3): La ciudad de Chernóbil fue evacuada el 27 de abril de 1986, un día después del accidente nuclear ...
La consulta fue: En español, qué ocurrió en Chernóbil? 

La respeusta fue: En Chernóbil ocurrió un accidente nuclear el 26 de abril de 1986 en la central nuclear Vladímir Ilich Len